In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.datasets import load_iris

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import os
os.getcwd()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [224]:
df = pd.read_csv("/home/saraiva/Documentos/Dataframes/travel-insurance/travel_insurance.csv")
df = load_iris()

X = pd.DataFrame(data = df.data,columns = df.feature_names)
y = pd.DataFrame(df.target)

In [225]:
param_grid  = {'bootstrap' :[True,False], 
               'max_depth' : list(range(2, 10)),
               'max_features' : ['auto','sqrt','log2'],
               'min_samples_leaf': list(range(5,20)), 
               'n_estimators': list(range(30, 300 ,5))}

In [226]:
algo = RandomForestClassifier
metric_list = [accuracy_score]

In [229]:
def random_objective(params,
                     algo,
                     metric_list,
                     X, y,
                     iterations = 20):
    """Random search objective function. Takes in hyperparameters
       and returns a list of results to be saved."""
    
    #Define as colunas do dataframe de saida
    param_col = [key for key in param_grid.keys()] + [metrics.__name__ for metrics in metric_list]
    
    lots_vals = list()
    for rounds in range(iterations):
        #Aleatoriza os parametros
        params = { key : random.sample(value,1)[0] for key,value in param_grid.items()}

        # Perform n_folds cross validation
        kfolds = KFold(n_splits=10,shuffle=True)

        matrix_metrics = []
        for train_idx,test_idx in kfolds.split(X):

            X_train,X_test = X.loc[train_idx,:],X.loc[test_idx,:]
            y_train,y_test = y.loc[train_idx,:],y.loc[test_idx,:]
            
            new_algo = algo(**params)

            new_algo.fit(X_train,np.ravel(y_train))
            y_pred = new_algo.predict(X_test)


            evaluation_list = []
            for evaluations in metric_list:
                result = evaluations(y_test,y_pred)
                evaluation_list.append(result)
            matrix_metrics.append(evaluation_list)

        #Dataframe criado com os resultados das metricas em cada fold
        aux_metrics = pd.DataFrame(matrix_metrics,columns = [metrics.__name__ for metrics in metric_list])
        metrics_results = aux_metrics.mean().tolist()


        # Return list of results
        vals = list(params.values()) + metrics_results
    
        lots_vals.append(vals)
    
    return pd.DataFrame(lots_vals,columns = param_col)

In [230]:
random_objective(params,algo,metric_list,X,y)

,bootstrap,max_depth,max_features,min_samples_leaf,n_estimators,accuracy_score
0,True,9,sqrt,14,85,0.953
1,False,9,auto,9,170,0.940
2,True,9,log2,11,170,0.947
3,False,7,auto,14,140,0.953
4,False,9,log2,16,115,0.947
5,True,2,log2,12,95,0.947
6,False,9,log2,7,105,0.940
7,False,2,auto,14,135,0.927
8,True,2,sqrt,11,95,0.960
9,True,9,log2,5,105,0.953
